In [1]:
# Importing dependencies
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Reading CSV files to begin cleaning & merging
cab_ride=pd.read_csv("Cab-Weather-Data/cab_rides.csv")
weather_df=pd.read_csv("Cab-Weather-Data/weather.csv")

In [3]:
# Cleaning Weather Data
# The timestamps were listed in epoch formatting, and needs to be converted to a readable date format

weather_df["date"]=pd.to_datetime(weather_df["time_stamp"], unit="s")

# Moving columns into a more readable way
clean_weather_df=weather_df[["date","location","temp","clouds","pressure","rain","humidity","wind"]]

# Sorting our weather table in ascending order per date time
clean_weather_df = clean_weather_df.sort_values("date", ascending=True)

# Reset the index and change the location header to source. This is so our merge will align on common time point & weather conditions at pick up location
new_weather_df = clean_weather_df.reset_index(drop=True)
new_weather_df.rename(columns={"location":"source"}, inplace=True)
new_weather_df.head()

,date,source,temp,clouds,pressure,rain,humidity,wind
0,2018-11-26 03:40:44,West End,40.84,0.87,1014.40,NaN,0.93,1.52
1,2018-11-26 03:40:44,Haymarket Square,40.98,0.87,1014.40,NaN,0.92,1.57
2,2018-11-26 03:40:44,South Station,40.86,0.87,1014.39,NaN,0.93,1.60
3,2018-11-26 03:40:44,Northeastern University,40.81,0.89,1014.35,NaN,0.93,1.36
4,2018-11-26 03:40:45,Back Bay,41.04,0.87,1014.39,NaN,0.92,1.46


In [4]:
# Cleaning Car Ride Data
# The epoch timestamp in this table is represented in milliseconds, so we divide by 1000 to find the time in value of seconds
cab_ride["date"]=pd.to_datetime(cab_ride["time_stamp"]/1000, unit="s")

# Removing unwanted columns like id and product_id
clean_cab_df = cab_ride[["date","source","destination","distance","price","surge_multiplier","cab_type", "name"]]

# In order to align on the datetime format used in the weather data, we are using a function that changes it into YYYY-MM-DD H:M:S format
clean_cab_df["date"]=clean_cab_df["date"].dt.strftime("%Y-%m-%d %H:%M:%S")
clean_cab_df = clean_cab_df.sort_values("date", ascending=True)

# Resetting the index
new_cab_df=clean_cab_df.reset_index(drop=True)
new_cab_df.head()

/var/folders/33/k2j4j20s7fb5j52jx4p6qr0w0000gn/T/ipykernel_45987/1249783199.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_cab_df["date"]=clean_cab_df["date"].dt.strftime("%Y-%m-%d %H:%M:%S")


,date,source,destination,distance,price,surge_multiplier,cab_type,name
0,2018-11-26 03:40:46,North Station,Haymarket Square,0.56,7.0,1.0,Uber,UberX
1,2018-11-26 03:40:46,Theatre District,North End,1.57,NaN,1.0,Uber,Taxi
2,2018-11-26 03:40:46,North End,West End,1.23,7.0,1.0,Lyft,Lyft
3,2018-11-26 03:40:46,Boston University,Beacon Hill,2.66,19.5,1.0,Lyft,Lux
4,2018-11-26 03:40:46,North End,West End,1.23,5.0,1.0,Lyft,Shared


In [5]:
# Changing data type to ensure easy merging
new_cab_df["date"] = new_cab_df["date"].astype("datetime64[ns]")
print(new_cab_df.dtypes)

date                datetime64[ns]
source                      object
destination                 object
distance                   float64
price                      float64
surge_multiplier           float64
cab_type                    object
name                        object
dtype: object


In [6]:
# Merging both dataframes on Date & Source. Left Join indicates that we only want to retain weather conditions for active rides

combined_df=pd.merge(new_cab_df,new_weather_df, on=["date","source"], how="left")
combined_df.head()

,date,source,destination,distance,price,surge_multiplier,cab_type,name,temp,clouds,pressure,rain,humidity,wind
0,2018-11-26 03:40:46,North Station,Haymarket Square,0.56,7.0,1.0,Uber,UberX,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-11-26 03:40:46,Theatre District,North End,1.57,NaN,1.0,Uber,Taxi,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-11-26 03:40:46,North End,West End,1.23,7.0,1.0,Lyft,Lyft,41.02,0.87,1014.4,NaN,0.92,1.59
3,2018-11-26 03:40:46,Boston University,Beacon Hill,2.66,19.5,1.0,Lyft,Lux,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-11-26 03:40:46,North End,West End,1.23,5.0,1.0,Lyft,Shared,41.02,0.87,1014.4,NaN,0.92,1.59


In [7]:
# To ensure we have all rows that have weather conditions present, we want to drop all null values in the Temperature column
# Null values for rain or wind is ok because these are logical weather conditions
final_df=combined_df.dropna(subset=["temp"])
final_df=final_df.reset_index(drop=True)

# With the null values in other rows, we can fill them with "0"
final_df.fillna(0,inplace=True)
final_df.head()

,date,source,destination,distance,price,surge_multiplier,cab_type,name,temp,clouds,pressure,rain,humidity,wind
0,2018-11-26 03:40:46,North End,West End,1.23,7.0,1.0,Lyft,Lyft,41.02,0.87,1014.40,0.0,0.92,1.59
1,2018-11-26 03:40:46,North End,West End,1.23,5.0,1.0,Lyft,Shared,41.02,0.87,1014.40,0.0,0.92,1.59
2,2018-11-26 03:40:46,North End,West End,1.23,13.5,1.0,Lyft,Lux,41.02,0.87,1014.40,0.0,0.92,1.59
3,2018-11-26 03:40:46,North End,West End,1.23,19.5,1.0,Lyft,Lux Black,41.02,0.87,1014.40,0.0,0.92,1.59
4,2018-11-26 03:40:46,Fenway,Theatre District,2.96,11.0,1.0,Lyft,Lyft,40.84,0.88,1014.35,0.0,0.93,1.31


In [8]:
# Adding the day of the week to our table to see if that may have any correlation to weather
final_df["day of week"]=final_df["date"].dt.day_name()
final_df.head()

,date,source,destination,distance,price,surge_multiplier,cab_type,name,temp,clouds,pressure,rain,humidity,wind,day of week
0,2018-11-26 03:40:46,North End,West End,1.23,7.0,1.0,Lyft,Lyft,41.02,0.87,1014.40,0.0,0.92,1.59,Monday
1,2018-11-26 03:40:46,North End,West End,1.23,5.0,1.0,Lyft,Shared,41.02,0.87,1014.40,0.0,0.92,1.59,Monday
2,2018-11-26 03:40:46,North End,West End,1.23,13.5,1.0,Lyft,Lux,41.02,0.87,1014.40,0.0,0.92,1.59,Monday
3,2018-11-26 03:40:46,North End,West End,1.23,19.5,1.0,Lyft,Lux Black,41.02,0.87,1014.40,0.0,0.92,1.59,Monday
4,2018-11-26 03:40:46,Fenway,Theatre District,2.96,11.0,1.0,Lyft,Lyft,40.84,0.88,1014.35,0.0,0.93,1.31,Monday


In [9]:
# Formatting some columns to show accurate values
final_df["clouds"]=final_df["clouds"]*100
final_df["humidity"]=final_df["humidity"]*100

In [10]:
# Pretty-fying our final dataframe by including units of measurement for easy analysis

final_df = final_df.rename(columns={"date":"Date","day of week":"Weekday","source":"Pick Up Location","destination":"Destination",
                                    "distance":"Distance (m)","price":"Price ($)","surge_multiplier":"Surge Multiplier",
                                    "cab_type":"Rideshare App","name":"Ride Type","temp":"Temp (C)",
                                    "clouds":"Cloudiness (%)","pressure":"Pressure (MB)","rain":"Rainfall (in)",
                                    "humidity":"Humidity (%)","wind":"Wind Speed (mph)"})

final_df = final_df[["Date","Weekday","Pick Up Location","Destination","Distance (m)","Price ($)","Surge Multiplier","Rideshare App","Ride Type",
                     "Temp (C)","Cloudiness (%)","Pressure (MB)","Rainfall (in)","Humidity (%)","Wind Speed (mph)"]]
final_df.head()

,Date,Weekday,Pick Up Location,Destination,Distance (m),Price ($),Surge Multiplier,Rideshare App,Ride Type,Temp (C),Cloudiness (%),Pressure (MB),Rainfall (in),Humidity (%),Wind Speed (mph)
0,2018-11-26 03:40:46,Monday,North End,West End,1.23,7.0,1.0,Lyft,Lyft,41.02,87.0,1014.40,0.0,92.0,1.59
1,2018-11-26 03:40:46,Monday,North End,West End,1.23,5.0,1.0,Lyft,Shared,41.02,87.0,1014.40,0.0,92.0,1.59
2,2018-11-26 03:40:46,Monday,North End,West End,1.23,13.5,1.0,Lyft,Lux,41.02,87.0,1014.40,0.0,92.0,1.59
3,2018-11-26 03:40:46,Monday,North End,West End,1.23,19.5,1.0,Lyft,Lux Black,41.02,87.0,1014.40,0.0,92.0,1.59
4,2018-11-26 03:40:46,Monday,Fenway,Theatre District,2.96,11.0,1.0,Lyft,Lyft,40.84,88.0,1014.35,0.0,93.0,1.31


In [11]:
# Sending this dataframe into a csv file to analyze further

final_df.to_csv("rideshare_vs_weather.csv", index=False)